In [352]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### **1. Read Data**

In [353]:
# First, read the data.
df = pd.read_csv('/kaggle/input/fish-market/Fish.csv')
df.head()

### **2. Data Cleaning**

#### **a. Fix Null Data**

**Before checking for null data, it is necessary to replace data with a value of 0 to np.NaN, because it is impossible for data to contain weight, length1, length2, length3, height, width data which are 0.**

In [354]:
df.replace(0, np.NaN, inplace=True)
df.isna().sum()

**There is empty data in the Weight column. Where the data column in the Weight column is 0**

In [355]:
# Clear the empty data in the Weight column using the average weight of the fish species in the null data
df[df['Weight'].isna() == True]

In [356]:
# fix null data
df['Weight'].fillna(160.052632, inplace=True)

In [357]:
# sns.boxplot(x=df[cols], ax=ax)

#### **b. Check Outliers**

In [358]:
# We will try to analyze outliers using boxplot from seaborn
cols_numeric = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']
fig, axes = plt.subplots(2,3, figsize=(15, 10))

for cols, ax in zip(cols_numeric, axes.flatten()):
    ax.set_title(f'Outliers of {cols}', fontsize=10, pad=10)
    sns.boxplot(x=df[cols], ax=ax)

**There are outliers in the data column Weight, Length1, Length2 and Length3**

In [359]:
# Create a remove_outliers function
def remove_outliers(data):
    Q1, Q3 = data.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    upper_range = Q3 + 1.5*IQR
    lower_range = Q1 - 1.5*IQR
    return lower_range, upper_range

for i in cols_numeric:
    lower, upper = remove_outliers(df[i])
    
    df[i] = np.where(df[i] > upper, np.NaN, df[i])
    df[i] = np.where(df[i] < lower, np.NaN, df[i])
    df[i].dropna(axis= 0, inplace=True)

### **3. Exploratory Data Analyst (EDA)**

In [360]:
# I will show the most orders by Fish Species.
plt.figure(figsize=(8, 6))
plt.title('The Most Ordered Fish by Consumers', fontsize=15, pad=15)
sns.countplot(df.Species, edgecolor='black')
plt.grid(color='black', alpha=0.2)
plt.tight_layout()
plt.show()

The graph shows that the Perch fish species are in great demand by consumers. 

Next, let's try to analyze the reason why Perch fish species are in high demand by consumers through the average weight, length1, length2, length3, height and width.

In [361]:
# I try to divide the mean and standard deviation per species
mean_species = []
std_species = []
species = [df.Species.unique()]

for i in df.Species.unique():
    mean_species.append(df[df['Species'] == i].mean())
    std_species.append(df[df['Species'] == i].std())

    
# Next I will separate by weight, length1, length2, length3, height and width of each list.
mean_weight = []
std_weight = []

mean_height = []
std_height = []

mean_width = []
std_width = []

for i in range(0, 7):
    for k in range(0, 6):
        if k == 0:
            mean_weight.append(mean_species[i][k])
            std_weight.append(std_species[i][k])
        elif k == 4:
            mean_height.append(mean_species[i][k])
            std_height.append(std_species[i][k])
        elif k == 5:
            mean_width.append(mean_species[i][k])
            std_width.append(std_species[i][k])
            
mean_data = [mean_weight, mean_height, mean_width]
std_data = [std_weight, std_height, std_width]
cols = ['Weight', 'Height', 'Width']
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink']

In [362]:
fig, axes = plt.subplots(1, 3, figsize=(30, 5))

for k, mean_i, std_i in zip(range(0, 4), mean_data, std_data):
    if cols[k] == 'Weight':
        axes[k].set_title(f'Mean of {cols[k]} Species', fontsize=15, pad = 15)
        axes[k].bar(x=df.Species.unique(), height=mean_i, yerr=std_i, edgecolor='black', color=colors)
        axes[k].grid(color='black', linestyle=':')
        axes[k].set_ylabel(f'{cols[k]} (in g)', fontsize=15)
        axes[k].set_xlabel('Species', fontsize=15, labelpad=15)
    else:
        axes[k].set_title(f'Mean of {cols[k]} Species', fontsize=15, pad = 15)
        axes[k].bar(x=df.Species.unique(), height=mean_i, yerr=std_i, edgecolor='black', color=colors)
        axes[k].grid(color='black', linestyle=':')
        axes[k].set_ylabel(f'{cols[k]} (in cm)', fontsize=15)
        axes[k].set_xlabel('Species', fontsize=15, labelpad=15)

Species Perch, which is the highest heating of 56, has quite a variety of weight data and has an average weight of 382.23 grams with an average height of 7.86 cm, followed by a width of 4.74 cm. Then the Species Bream which is the second highest heating of 35, has a weight data that is also quite diverse with an average weight of 617.82 grams then has an average height of 15.18 cm and has a width of 5.42 cm. Then the Species Roach with the third highest order of 20 has an average weight of 152.05 grams with an average height of 6.69 cm and an average width of 3.65 cm.

From the data, it shows that the most ordering effect on Species is not influenced by the weight of a fish. However, it is influenced by the width of the Species. because the top 3 Species have a fairly large width compared to other Species. However, this decision may not be right because there are other factors that cause the number of orders based on species, namely the taste of the fish species which is relative to each consumer.